In [ ]:
__copyright__ = "Reiner Lemoine Institut gGmbH"
__license__ = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__ = "https://github.com/openego/eDisGo/blob/master/LICENSE"
__author__ = "mltja"

# eDisGo plot example with plotly

## Setup

#### Import packages

In [ ]:
import logging
logging.basicConfig(level=logging.CRITICAL)

In [ ]:
import os
import sys
import pandas as pd
import requests
import copy
import time

from edisgo import EDisGo
from edisgo.tools.plots import plot_plotly
from edisgo.tools.plots import plot_dash

#### Download example grid

In [ ]:
import requests

def download_ding0_example_grid():

    # create directories to save ding0 example grid into
    ding0_example_grid_path = os.path.join(
        os.path.expanduser("~"), ".edisgo", "ding0_test_network"
    )
    os.makedirs(ding0_example_grid_path, exist_ok=True)

    # download files
    filenames = [
        "buses",
        "generators",
        "lines",
        "loads",
        "network",
        "switches",
        "transformers",
        "transformers_hvmv",
    ]

    for file in filenames:
        req = requests.get(
            "https://raw.githubusercontent.com/openego/eDisGo/dev/tests/data/ding0_test_network_1/{}.csv".format(
                file
            )
        )
        filename = os.path.join(ding0_example_grid_path, "{}.csv".format(file))
        with open(filename, "wb") as fout:
            fout.write(req.content)


download_ding0_example_grid()

#### Create edisgo objects

In [ ]:
ding0_grid = os.path.join(os.path.expanduser("~"), ".edisgo", "ding0_test_network")

In [ ]:
edisgo_root = EDisGo(ding0_grid=ding0_grid)
edisgo_root.set_time_series_worst_case_analysis()

In [ ]:
edisgo_analyzed = copy.deepcopy(edisgo_root)
edisgo_analyzed.analyze();

In [ ]:
edisgo_reinforced = copy.deepcopy(edisgo_root)
edisgo_reinforced.reinforce();

## Plots

### plot_plotly function

In the following different plotting options are shown. For more information on different plotting options see [API docstring](https://edisgo.readthedocs.io/en/dev/api/edisgo.tools.html#edisgo.tools.plots.draw_plotly).


Hovering over nodes and lines shows some information on them.

#### Plotting relative loading and voltage deviation, with map in the background. 

Be aware that the used grid is a test grid with random geolocations. To see the plotting on a map with a ding0 grid with actual geolocations you can change the downloaded grid in the function `download_ding0_example_grid` above to 'ding0_test_network_3' and import the grid as follows:

```
edisgo_root = EDisGo(ding0_grid=ding0_grid, legacy_ding0_grids=False)
```

In [ ]:
fig = plot_plotly(
    edisgo_obj=edisgo_analyzed,
    grid=None,
    line_color="relative_loading",
    node_color="voltage_deviation",
    line_result_selection="max",
    node_result_selection="max",
    selected_timesteps=None,
    pseudo_coordinates=False,
    plot_map=True,
    node_selection=False
)

fig.show()

#### Plotting relative loading and voltage deviation, with time step selection

In [ ]:
plot_plotly(
    edisgo_obj=edisgo_analyzed,
    grid=None,
    line_color="relative_loading",
    node_color="voltage_deviation",
    line_result_selection="max",
    node_result_selection="max",
    selected_timesteps=[
        '1970-01-01 00:00:00', 
        '1970-01-01 01:00:00',
    ],
    pseudo_coordinates=False,
    node_selection=False
)

#### Plotting loading and voltage deviation, with pseudo coordinates

In [ ]:
plot_plotly(
    edisgo_obj=edisgo_analyzed,
    grid=None,
    line_color="loading",
    node_color="voltage_deviation",
    line_result_selection="max",
    node_result_selection="max",
    selected_timesteps='1970-01-01 03:00:00',
    pseudo_coordinates=True,
    node_selection=False
)

#### Plotting reinforced lines, node adjacencies and a node selection

In [ ]:
plot_plotly(
    edisgo_obj=edisgo_reinforced,
    grid=None,
    line_color="reinforce",
    node_color="adjacencies",
    line_result_selection="max",
    node_result_selection="max",
    selected_timesteps=None,
    pseudo_coordinates=False,
    node_selection=[
        'Bus_MVStation_1',
        'Bus_BranchTee_MVGrid_1_5',
        'Bus_BranchTee_MVGrid_1_10',
        'Bus_GeneratorFluctuating_4',
        'Bus_BranchTee_MVGrid_1_11',
        'Bus_GeneratorFluctuating_3',
        'BusBar_MVGrid_1_LVGrid_4_MV'
    ]
)

### Dash plot app which calls plot_plotly
One edisgo object creates one large plot. Two or more edisgo objects create two adjacent plots, where the objects to be plotted are selected in the dropdown menu.

#### One interactive plot

In [ ]:
plot_dash(edisgo_objects=edisgo_analyzed)
time.sleep(2)

#### Two interactives plots side by side

In [ ]:
plot_dash(edisgo_objects={"edisgo_analyzed": edisgo_analyzed, "edisgo_reinforced": edisgo_reinforced})
time.sleep(2)

#### Two interactives plots side by side with 3 EDisGo objects

Choose your edisgo objects in the drop down menu.

In [ ]:
plot_dash(
    edisgo_objects={
        "edisgo_root": edisgo_root,
        "edisgo_analyzed": edisgo_analyzed,
        "edisgo_reinforced": edisgo_reinforced
    }, height=1000
)